# Data Generation Notebook. 

This is the notebook for getting the data. First we collect the cluster data, afterwards the data for the forecasting and the validation set.

We have the following structure: 
#### 1. Generate Datasets Clustering
#### 2. Generate Dataset for Forecasting
#### 3. Get Test Data from Validation Environment

In [1]:
import os
#os.chdir("..")
print(os.getcwd())
os.cpu_count()

/home/mlehna/AI2Go/Paper


128

In [2]:
import json
import csv
from tqdm import tqdm
import grid2op

import numpy as np
import os
from pathlib import Path
from joblib import Parallel, delayed
import multiprocessing as mp

from grid2op.PlotGrid import PlotMatplot
from tqdm import tqdm
import pickle

/share/data1/GYM/anaconda3/envs/py310/lib/python3.10/site-packages/lightsim2grid/securityAnalysis.py:76: FutureWarning: In the future `np.str` will be defined as the corresponding NumPy scalar.
  STR_TYPES = (str, np.str, np.str_)


In [3]:
np.random.seed(8888)
seeds = list(np.random.randint(0, 10000,10))
print(seeds)

[4483, 2120, 6825, 5612, 2224, 4735, 1016, 3891, 2377, 4582]


In [4]:
env = grid2op.make('l2rpn_wcci_2022')
#plot_helper = PlotMatplot(env.observation_space)
obs_original = env.reset()
#fig = plot_helper.plot_obs(obs_original,storage_info=None,gen_info=None,load_info=None)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



# 1. Generate Datasets Clustering

In [19]:
# There are only 3 Error Types: 
err_id = {'Grid2OpException DivergingPowerFlow "Divergence of DC powerflow (non connected grid) at the initialization of AC powerflow. Detailed error: ErrorType.SolverFactor"': 0,
          'Grid2OpException DivergingPowerFlow "Divergence of AC powerflow. Detailed error: ErrorType.TooManyIterations"': 1,
          'Grid2OpException DivergingPowerFlow "Divergence of DC powerflow (non connected grid) at the initialization of AC powerflow. Detailed error: ErrorType.SolverSolve"':2}

def gen_obs(path):
    """
    Generate Observations
    """
    path = Path(path)
    metapath = path/"episode_meta.json"
    try: 
        with open(metapath) as json_file:
            meta = json.load(json_file)


        if meta['nb_timestep_played']==2017: 
            return None
        else: 
            try: 
                o = np.load(path /"observations.npz")["data"]
            except FileNotFoundError:
                pass
            # Get error type: 
            e_t = meta['info'].get('exception',["no_error"])[0]

            # Set counter & id
            id_e = err_id[e_t]

            # Extract and save observation step -1,-3,-5
            obs_start = np.hstack([-1,0,o[0]])
            obs1 = np.hstack([id_e,meta['nb_timestep_played'],o[meta['nb_timestep_played']-1]])
            obs3 = np.hstack([id_e,meta['nb_timestep_played'],o[meta['nb_timestep_played']-3]])
            obs5 = np.hstack([id_e,meta['nb_timestep_played'],o[meta['nb_timestep_played']-5]])
            return [obs1,obs3,obs5,obs_start]
    except FileNotFoundError:
        logging.info(f"Meta not found in {path}")
        return None

    except json.JSONDecodeError:
        logging.info(f"Meta had a problem loading: {metapath}")
        return None
    

In [20]:
gen_obs(parallel_set[0])

[array([1.00e+00, 6.76e+02, 2.05e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00]),
 array([1.00e+00, 6.76e+02, 2.05e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00]),
 array([1.00e+00, 6.76e+02, 2.05e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00]),
 array([-1.00000000e+00,  0.00000000e+00,  2.05000000e+03, ...,
         0.00000000e+00,  5.69999981e+00,  0.00000000e+00])]

In [33]:
import logging
from multiprocessing import Pool
agent_res =  {}

for agent in ["Senior_original_95","DoNothing","Topo_Agent_95_2"]: #,"Senior_original_95", #,"DoNothing","Senior_original_95"
    print(f"Collecting {agent}")
    obs1_res =[]
    obs3_res = []
    obs5_res = []
    obs_start_r = []
    for seed in tqdm(seeds):
        path_saved = Path(f"/share/data1/GYM/seed_ecml2/{seed}/agent_logs/{agent}/")
        if path_saved.is_dir():
            all_ep = [ep for ep in os.listdir(Path(f"/share/data1/GYM/seed_ecml2/{seed}/agent_logs/{agent}/"))  if "2050" in ep]
            parallel_set = [str(path_saved / ep) for ep in all_ep]
            print(f"Collecting all {len(parallel_set)} episodes of seed {seed} for agent {agent}:")
            
            # ToDo: Pooling:
            with Pool() as pool:
                out = pool.map(gen_obs, parallel_set)
            
            out = [o for o in out if o is not None]
        
        for o in out:
            obs1_res.append(o[0])
            obs3_res.append(o[1])
            obs5_res.append(o[2])
            obs_start_r.append(o[3])
        print(f"Done with Seed {seed}. A total of {np.round(len(out)/len(parallel_set)*100,0)}% could be processed")
    
    agent_res[agent] = [obs1_res,obs3_res,obs5_res,obs_start_r]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:21<03:16, 21.79s/it]

Done with Seed 4483. A total of 71.0% could be processed


 20%|██        | 2/10 [00:44<02:56, 22.07s/it]

Done with Seed 2120. A total of 71.0% could be processed


...

Done with Seed 3891. A total of 68.0% could be processed


 90%|█████████ | 9/10 [03:48<00:26, 26.17s/it]

Done with Seed 2377. A total of 70.0% could be processed


100%|██████████| 10/10 [04:13<00:00, 25.35s/it]

Done with Seed 4582. A total of 68.0% could be processed


In [38]:
# Store data (serialize)
with open('/share/data1/GYM/evaltrain_full.pkl', 'wb') as handle:
    pickle.dump(agent_res, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load data (deserialize)
with open('/share/data1/GYM/evaltrain_full.pkl', 'rb') as handle:
    agent_res = pickle.load(handle)

# 2. Generate Dataset for Forecasting

In [71]:

def get_ep_res(path, top_n = 12):
    """
    Generate episode results
    """
    path = Path(path)
    obs_data = []
    try:
        with open(path/ "episode_meta.json") as json_file:
            meta = json.load(json_file)

        if not meta['nb_timestep_played'] == 2017 and meta['nb_timestep_played'] > 10: 
            o = np.load(path/ "observations.npz")["data"]

            if len(o) <= 5+(top_n*10)+1:
                raise Exception(f"Too short observation list in ep {ep}")

            # Extract and save bad observations at steps -1, -3, -5
            obs_data.append(np.append(o[meta['nb_timestep_played']-1],[1]))
            obs_data.append(np.append(o[meta['nb_timestep_played']-3],[2]))
            obs_data.append(np.append(o[meta['nb_timestep_played']-5],[3]))

            # Extract all observations from 0 to max-6
            observations = [(i, env.observation_space.from_vect(o[i])) for i in range(meta['nb_timestep_played'] - 6)]

            # Sort them based on max_rho
            sorted_obs_with_ids = sorted(observations, key=lambda x: x[1].rho.max(), reverse=True)

            # Sample the top good observation
            top_observation = sorted_obs_with_ids[0]

            remaining_observations = sorted(sorted_obs_with_ids[1:top_n*10], key=lambda x: x[0], reverse=True)  # Sort by index
            sampled_observations = [top_observation]

            for candidate in remaining_observations:
                if len(sampled_observations) >= top_n:
                    break
                if all(abs(candidate[0] - obs[0]) >= 10 for obs in sampled_observations):
                    sampled_observations.append(candidate)

            # Append the sampled good observations to obs_data
            for id, _ in sampled_observations:
                obs_data.append(np.append(o[id], [0]))
            
            return obs_data
                      
    except Exception as e: 
        print(f"Error with agent in path {path}: {e}")
    
    
def generate_datasets2(seeds_path, savepath, environment_name, seed_ids=None, ep_year="2050", percentage_bad=0.2):
    full_data = []
    for agent in ["Topo_Agent_95_2", "Senior_original_95", "DoNothing"]:
        
        # calculate how many good observations regarding percentage_bad parameter
        top_n = int(np.round(3*(1-percentage_bad)/percentage_bad))
        print(f"Generating of datasets with distribution: game_over={percentage_bad}, not_game_over={1-percentage_bad}")
        env = grid2op.make(environment_name)

        obs_data = []
        print(f"Collecting {agent}")
        
        seeds = seed_ids if seed_ids is not None else list(np.random.randint(0, 10000, 10))
        for seed in tqdm(seeds):
            path_saved = Path(f"{seeds_path}/{seed}/agent_logs/{agent}/")
            
            all_ep = [ep for ep in os.listdir(path_saved) if str(ep_year) in ep]
            print(f"Collecting all {len(all_ep)} episodes of seed {seed} for agent {agent}:")
            
            parallel_set = [(path_saved / ep ,top_n) for ep in all_ep]
            
            with Pool() as pool:
                out = pool.starmap(get_ep_res, parallel_set)
            
            out = [o for o in out if o is not None]
            
            obs_data.append(np.concatenate(out))
        
        # Save for agent: 
        obs_data = np.concatenate(obs_data)
        
        obs_data = np.unique( obs_data, axis=0)
        print(f"Save agent results of shape {obs_data.shape}")
        
        
        full_data.append(obs_data)
        # Save all observations as an array dumped to csv
        np.save(savepath/ f"{agent}_obs_data.npy",obs_data)

   # Save for all: 
    full_data = np.concatenate(full_data)
    # Save all observations as an array dumped to csv
    np.save(savepath/ f"all_obs_data.npy",full_data)


In [73]:
generate_datasets2("/share/data1/GYM/seed_ecml2/", Path("/share/data1/GYM/"), 'l2rpn_wcci_2022', seed_ids=seeds, ep_year="2050", percentage_bad=0.5)

Generating of datasets with distribution: game_over=0.5, not_game_over=0.5


Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [01:32<13:53, 92.60s/it]

 20%|██        | 2/10 [03:08<12:35, 94.50s/it]

...

 70%|███████   | 7/10 [11:50<05:12, 104.20s/it]

 80%|████████  | 8/10 [13:33<03:27, 103.59s/it]

 90%|█████████ | 9/10 [15:17<01:43, 103.81s/it]

100%|██████████| 10/10 [17:01<00:00, 102.13s/it]


Save agent results of shape (74607, 4296)


In [74]:
senior = np.load("/share/data1/GYM/Senior_original_95_obs_data.npy")
topo = np.load("/share/data1/GYM/Topo_Agent_95_2_obs_data.npy")
dn = np.load("/share/data1/GYM/DoNothing_obs_data.npy")

In [75]:
print(senior.shape)
senior_sub = np.unique(senior, axis=0)
print(senior_sub.shape)
print()

print(topo.shape)
topo_sub = np.unique(topo, axis=0)
print(topo_sub.shape)
print()
print(dn.shape)
dn_sub = np.unique(dn, axis=0)
print(dn_sub.shape)
print()

(59016, 4296)
(59016, 4296)

(55772, 4296)
(55772, 4296)

(74607, 4296)
(74607, 4296)



In [76]:
np.savez_compressed('/share/data1/GYM/full_obs_data.npz', senior=senior_sub, topo=topo_sub,dn=dn_sub)

# 3. Get Test Data from Validation Environment

In [77]:
generate_datasets2("/share/data1/GYM/seed_ecml_val/", Path("/share/data1/GYM/seed_ecml_val/"), "/share/data1/GYM/data_grid2op/l2rpn_2022_val_6987/", seed_ids=seeds, ep_year="2050", percentage_bad=0.5)

Generating of datasets with distribution: game_over=0.5, not_game_over=0.5


Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:57<08:38, 57.65s/it]

 20%|██        | 2/10 [01:59<07:59, 59.92s/it]

...

 70%|███████   | 7/10 [08:05<03:26, 68.70s/it]

 80%|████████  | 8/10 [09:04<02:11, 65.69s/it]

 90%|█████████ | 9/10 [10:04<01:03, 63.85s/it]

100%|██████████| 10/10 [11:03<00:00, 66.37s/it]


Save agent results of shape (2361, 4296)
